<a href="https://colab.research.google.com/github/devashishbotre/Autonomous-Lane-Detector/blob/main/scripts/Lane_Detection_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
manideep1108_tusimple_path = kagglehub.dataset_download('manideep1108/tusimple')

print('Data source import complete.')


In [ ]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from scipy.interpolate import CubicSpline
from sklearn.cluster import DBSCAN
import torchvision.models as models

# Constants
IMG_HEIGHT = 720
IMG_WIDTH = 1280
NUM_CLASSES = 2  # Binary: 0 (background), 1 (lane)

In [ ]:
class RESA(nn.Module):
    def __init__(self, in_channels):
        super(RESA, self).__init__()
        # Reduce kernel size from 3x3 to 1x1 for finer detail preservation
        self.conv = nn.Conv2d(in_channels, in_channels, 1, padding=0)
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        # Add an extra layer for feature refinement
        self.refine_conv = nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels)
        self.refine_bn = nn.BatchNorm2d(in_channels)
        self.shifts = [
            (1, 0), (2, 0), (3, 0),    # up-to-down
            (-1, 0), (-2, 0), (-3, 0), # down-to-up
            (0, -1), (0, -2), (0, -3), # right-to-left
            (0, 1), (0, 2), (0, 3)     # left-to-right
        ]

        distances = torch.tensor([(i**2 + j**2) for i, j in self.shifts], dtype=torch.float32)
        self.weights = torch.exp(-distances / (2 * 1.5**2)).to(device)

    def forward(self, x):
        contributions = []
        for (shift_h, shift_w), weight in zip(self.shifts, self.weights):
            shifted = torch.roll(x, shifts=(shift_h, shift_w), dims=(2, 3))
            contrib = self.relu(self.bn(self.conv(shifted)))
            contrib = self.relu(self.refine_bn(self.refine_conv(contrib)))
            contributions.append(contrib * weight)

        # Weighted sum with original input
        out = x + 0.5 * sum(contributions) / sum(self.weights)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.Up1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 2, 3, padding=1),
            nn.BatchNorm2d(in_channels // 2),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        self.Up2 = nn.Sequential(
            nn.Conv2d(in_channels // 2, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

        self.smooth_conv = nn.Conv2d(out_channels, out_channels, 5, padding=2, groups=out_channels)
        self.smooth_bn = nn.BatchNorm2d(out_channels)

        # Add thinning layers
        self.thin1 = nn.Conv2d(in_channels // 2, in_channels // 2, 3, padding=1, groups=in_channels // 2)
        self.thin2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, groups=out_channels)

        # Define ReLU as an attribute
        self.relu = nn.ReLU(inplace=True)  # Add this line

    def forward(self, x):
        x = self.Up1(x)
        x = x + torch.sigmoid(self.thin1(x)) * x  # Focus attention on thin structures
        x = self.Up2(x)
        x = x + torch.sigmoid(self.thin2(x)) * x  # Further refine thin structures

        # Apply smoothing using self.relu
        x = self.relu(self.smooth_bn(self.smooth_conv(x)))
        return x

In [ ]:
class LaneNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, k_iterations=4):
        super(LaneNet, self).__init__()
        resnet = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        self.encoder = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        )  # Output: [batch, 512, H/32, W/32]

        self.resa_layers = nn.ModuleList([RESA(512) for _ in range(k_iterations)])
        self.decoder = Decoder(512, 256)
        self.seg_head = nn.Conv2d(256, num_classes, 1)

    def forward(self, x):
        x = self.encoder(x)
        for resa in self.resa_layers:
            x = resa(x)
        x = self.decoder(x)
        seg_out = self.seg_head(x)
        seg_out = F.interpolate(seg_out, size=(IMG_HEIGHT, IMG_WIDTH), mode='bilinear', align_corners=True)
        seg_out = F.avg_pool2d(seg_out, kernel_size=3, padding=1, stride=1)
        return seg_out

In [ ]:
class TuSimpleDataset(Dataset):
    def __init__(self, json_files, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.annotations = []
        total_annotations = 0
        for json_file in json_files:
            if not os.path.exists(json_file):
                print(f"Warning: JSON file not found: {json_file}")
                continue
            with open(json_file, 'r') as f:
                lines = f.readlines()
                total_annotations += len(lines)
                for line in lines:
                    ann = json.loads(line)
                    img_path = os.path.join(self.img_dir, ann['raw_file'])
                    if os.path.exists(img_path):
                        self.annotations.append(ann)
                    else:
                        print(f"Warning: Image not found: {img_path}")
        print(f"Total annotations in JSON: {total_annotations}")
        print(f"Valid images found: {len(self.annotations)}")

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img_path = os.path.join(self.img_dir, ann['raw_file'])

        # Load and resize image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error: Failed to load image: {img_path}")
            image = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)  # Fallback to black image
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Create mask
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
        for lane in ann['lanes']:
            points = [(x, y) for x, y in zip(lane, ann['h_samples']) if x != -2 and 0 <= y < IMG_HEIGHT and 0 <= x < IMG_WIDTH]
            if len(points) > 1:
                points = np.array(points, dtype=np.int32)
                cv2.polylines(mask, [points], False, 1, 1)

        # Convert to tensors
        image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
        mask = torch.from_numpy(mask).long()

        return image, mask, ann

# Custom collate function to handle tuples
def collate_fn(batch):
    images, masks, anns = zip(*batch)
    images = torch.stack(images, 0)  # Stack images into [batch_size, 3, H, W]
    masks = torch.stack(masks, 0)    # Stack masks into [batch_size, H, W]
    return images, masks, anns

In [ ]:
def train_model():
    batch_size = 4
    num_epochs = 20
    learning_rate = 0.0002
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LaneNet(num_classes=NUM_CLASSES).to(device)

    # Stronger class imbalance weighting
    weights = torch.tensor([1.0, 25.0]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    # Loss weights
    tv_weight = 0.3
    narrowness_weight = 0.5
    edge_weight = 0.2

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    data_dir = '/kaggle/input/tusimple/TUSimple/train_set'
    json_files = [os.path.join(data_dir, f) for f in ['label_data_0313.json',
                                                     'label_data_0531.json',
                                                     'label_data_0601.json']]
    img_dir = data_dir
    train_dataset = TuSimpleDataset(json_files, img_dir)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=collate_fn)

    # Assume train_loader is defined elsewhere
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, masks, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            seg_loss = criterion(outputs, masks)

            # Total Variation Loss
            def total_variation_loss(x):
                batch_size = x.size(0)
                h_x = x.size(2)
                w_x = x.size(3)
                dh = torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :]).sum() / (batch_size * h_x * w_x)
                dw = torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1]).sum() / (batch_size * h_x * w_x)
                return (dh + dw) / 2

            # Narrowness Penalty with clamping
            def narrowness_penalty(x):
                probs = torch.softmax(x, dim=1)
                lane_prob = probs[:, 1, :, :]  # Probability of lane class
                kernel = torch.ones(1, 1, 3, 3).to(device) / 9
                blurred = F.conv2d(lane_prob.unsqueeze(1), kernel, padding=1)

                # Clamp to avoid extreme values
                blurred = torch.clamp(blurred, min=1e-6, max=1.0 - 1e-6)
                lane_prob = torch.clamp(lane_prob, min=1e-6, max=1.0 - 1e-6)

                width_penalty = torch.pow(blurred.squeeze(1), 2).mean()
                variance = torch.pow(lane_prob - blurred.squeeze(1), 2).mean()
                return width_penalty + variance

            # Edge Enhancement Loss with stabilization
            def edge_enhancement_loss(x):
                probs = torch.softmax(x, dim=1)
                lane_prob = probs[:, 1, :, :]

                sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).to(device)
                sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).to(device)

                sobel_x = sobel_x.view(1, 1, 3, 3)
                sobel_y = sobel_y.view(1, 1, 3, 3)

                edge_x = F.conv2d(lane_prob.unsqueeze(1), sobel_x, padding=1)
                edge_y = F.conv2d(lane_prob.unsqueeze(1), sobel_y, padding=1)

                edges = torch.sqrt(edge_x.pow(2) + edge_y.pow(2) + 1e-8)  # Add epsilon for stability
                return -torch.clamp(edges.mean(), max=10.0)  # Cap the negative value

            # Compute losses
            tv_loss = total_variation_loss(torch.softmax(outputs, dim=1))
            narrowness_loss = narrowness_penalty(outputs)
            edge_loss = edge_enhancement_loss(outputs)

            total_loss = seg_loss + tv_weight * tv_loss + narrowness_weight * narrowness_loss + edge_weight * edge_loss

            # Check for NaN
            if torch.isnan(total_loss):
                print(f"NaN detected: seg_loss={seg_loss.item()}, tv_loss={tv_loss.item()}, "
                      f"narrowness_loss={narrowness_loss.item()}, edge_loss={edge_loss.item()}")
                break

            optimizer.zero_grad()
            total_loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            running_loss += total_loss.item()

        if torch.isnan(total_loss):
            break

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        if (epoch + 1) % 5 == 0:
            torch.save(model.state_dict(), f"lane_model_epoch_{epoch+1}.pth")

    torch.save(model.state_dict(), "lane_model_final.pth")
    print("Training completed!")
    return model, device

In [ ]:
model,device=train_model()

Total annotations in JSON: 3626
Valid images found: 3626


Epoch 1/20: 100%|██████████| 907/907 [06:13<00:00,  2.43it/s]


Epoch [1/20], Loss: 0.0869


Epoch 2/20: 100%|██████████| 907/907 [06:12<00:00,  2.43it/s]


Epoch [2/20], Loss: 0.0689


...

Epoch 8/20: 100%|██████████| 907/907 [06:13<00:00,  2.43it/s]


Epoch [8/20], Loss: 0.0518


Epoch 9/20: 100%|██████████| 907/907 [06:11<00:00,  2.44it/s]


Epoch [9/20], Loss: 0.0486


Epoch 10/20:  77%|███████▋  | 699/907 [04:45<01:24,  2.46it/s]